In [59]:
import cvxpy as cp
import numpy as np

N,M = 16,16
K = 2
H = np.random.randn(N,K) + 1j*np.random.randn(N,K)

noise_var = 2
min_snr = 1

c_1 = (1/np.sqrt(min_snr*noise_var))
c_2 = (1/noise_var)

W = cp.Variable((N,K), complex=True)
obj = cp.Minimize(cp.square(cp.norm(W)))

constraints = []
for k in range(K):
    mask = np.eye(K)
    mask[k,k] = 0
    constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*(W @ mask).H @ H[:,k], np.ones(1))), 2)]


    # constraints += [cp.SOC((A @ W[:,k]), cp.real(H[:,k].conj().T @ W[:,k]))]
prob = cp.Problem(obj, constraints)
prob.solve(verbose=False)
print(W.value)
print(np.linalg.norm(W.value, 2))

[[-0.01144787-0.01180469j -0.01559544-0.0032558j ]
 [ 0.02840486-0.02429711j -0.0023478 +0.04197601j]
 [-0.07357254+0.03371111j -0.01350595+0.11841373j]
 [-0.00015261-0.01618464j  0.0067697 +0.03383853j]
 [-0.04007321+0.06364236j  0.02661526+0.0089671j ]
 [ 0.04786561+0.05364511j -0.0623281 -0.00412571j]
 [-0.07832796-0.04161672j -0.069665  +0.02894552j]
 [-0.08115071+0.08236947j  0.00253385-0.09190328j]
 [ 0.01967615-0.02190635j  0.05210976-0.03164581j]
 [ 0.03082417+0.03671595j  0.05895479+0.03424878j]
 [-0.02761169-0.01847614j  0.06625997-0.1121177j ]
 [-0.01760637+0.01202843j -0.09067836+0.02479937j]
 [ 0.00150578-0.02660632j -0.00072486+0.06517021j]
 [-0.00357548-0.07619498j  0.0528799 -0.03504743j]
 [ 0.04086585-0.16538789j -0.03761601-0.00519723j]
 [-0.04457805-0.0290101j   0.07116224+0.00585452j]]
0.29193692448144604


In [2]:
import numpy as np
from as_bb import *

max_ant = 5
N = 8
M = 4
H = np.random.randn(N, M) + 1j*np.random.randn(N,M)    
instance = np.stack((np.real(H), np.imag(H)), axis=0)
max_iter=1000
policy='default'
policy_type='gnn'
oracle_opt=None

# def solve_bb(instance, max_ant, max_iter=1000, policy='default', policy_type='gnn', oracle_opt=None):
if policy_type == 'default':
    env = ASBBenv(observation_function=Observation, epsilon=0.001)
elif policy_type == 'gnn':
    env = ASBBenv(observation_function=Observation, epsilon=0.001)
elif policy_type == 'linear':
    env = ASBBenv(observation_function=LinearObservation, epsilon=0.001)
elif policy_type == 'oracle':
    env = ASBBenv(observation_function=Observation, epsilon=0.001)
    pass

branching_policy = DefaultBranchingPolicy()

t1 = time.time()

env.reset(instance, max_ant=max_ant)
timestep = 0
done = False
print(len(env.nodes))
while timestep < max_iter and len(env.nodes)>0 and not done:
    print('timestep', timestep, env.global_U, env.global_L)
    print(env.L_list)
    env.fathom_nodes()
    if len(env.nodes) == 0:
        break
    node_id, node_feats, label = env.select_node()

    if len(env.nodes) == 0:
        break
    # prune_node = env.prune(node_feats)
    # if prune_node:
    #     env.delete_node(node_id)
    #     continue
    # else:
    branching_var = branching_policy.select_variable(node_feats, env.action_set_indices)
    done = env.push_children(branching_var, node_id)
    timestep = timestep+1
print('ended')

1
timestep 0 inf 0.5153135184546469
[0.5153135184546469]
l left 0.6164019716552315
l right 0.5153135108297879
timestep 1 inf 0.5153135108297879
[0.6164019716552315, 0.5153135108297879]
l left 0.5691621487596467
l right 0.5153135134175285
timestep 2 inf 0.5153135134175285
[0.6164019716552315, 0.5691621487596467, 0.5153135134175285]
l left 0.5886514890679213
l right 0.5153135139706291
timestep 3 inf 0.5153135139706291
[0.6164019716552315, 0.5691621487596467, 0.5886514890679213, 0.5153135139706291]
l left 0.5860337796251753
l right 0.5153135739175363
timestep 4 5.602689421299707 0.5153135739175363
[0.6164019716552315, 0.5691621487596467, 0.5886514890679213, 0.5860337796251753, 0.5153135739175363]
l left 0.6348682351777972
l right  2.0339955663666736
timestep 5 2.0339955663666736 0.5691621487596467
[0.6164019716552315, 0.5691621487596467, 0.5886514890679213, 0.5860337796251753, 0.6348682351777972]
l left 0.6584121448537648
l right 0.5691622295704932
timestep 6 0.8826369419313004 0.56916222

l right  1.3445890703052568
timestep 31 0.8046931844793327 0.7166987604329199
[0.7278136862634497, 0.7262946799294896, 0.7377835698283166, 0.7166987604329199, 0.7429485888686012, 0.7774162273454238]
l right  1.4719410315567947
timestep 32 0.8046931844793327 0.7262946799294896
[0.7278136862634497, 0.7262946799294896, 0.7377835698283166, 0.7429485888686012, 0.7774162273454238]
l right 0.7262945795060755
timestep 33 0.8046931844793327 0.7262945795060755
[0.7278136862634497, 0.7377835698283166, 0.7429485888686012, 0.7774162273454238, 0.7262945795060755]
l right  1.779949432736898
timestep 34 0.8046931844793327 0.7278136862634497
[0.7278136862634497, 0.7377835698283166, 0.7429485888686012, 0.7774162273454238]
l right 0.7278136957177697
timestep 35 0.8046931844793327 0.7278136957177697
[0.7377835698283166, 0.7429485888686012, 0.7774162273454238, 0.7278136957177697]
l right  1.4776034549216706
timestep 36 0.8046931844793327 0.7377835698283166
[0.7377835698283166, 0.7429485888686012, 0.7774162

In [39]:
a = np.argwhere(np.array(env.L_list)>0.28)

In [40]:
if len(a)>0:
    for i in a.squeeze(axis=1):
        print(i)

0


# Parallel Processing

In [9]:
import numpy as np
from multiprocessing import Pool
import time
from as_bb import *

max_ant = 5
N = 8
M = 4
num_egs = 3
H = np.random.randn(num_egs, N, M) + 1j*np.random.randn(num_egs, N,M)    
instance = np.stack((np.real(H), np.imag(H)), axis=1)
max_iter=1000
policy='default'
policy_type='gnn'
oracle_opt=None

class BBParallel(object):
    def __init__(self, max_ant, max_iter=1000, policy='default', policy_type='gnn', oracle_opt=None):
        self.max_ant = max_ant
        self.max_iter = max_iter
        self.policy = policy
        self.policy_type = policy_type
        self.oracle_opt = oracle_opt
        
    def pool_process(self, tup):
        instance, iteration = tup
        return solve_bb(instance, self.max_ant)
        

BBpar_instance = BBParallel(max_ant)


t1 = time.time()

for i in range(4):
    with Pool(30) as p:
        out = p.map(BBpar_instance.pool_process, list(zip(list(instance), range(0, len(instance)))))
    
print('time ', time.time()-t1)    



/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb.py:364: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


1
timestep 0 inf 0.5342319573135668


/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb.py:364: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


1
timestep 0 inf 0.3103712106221212


/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb.py:364: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


...

1
timestep 0 inf 0.6129251568180112
timestep 1 inf 0.3103712645068246
timestep 1 timestepinf  10.5342319390717347
 inf 0.6129252961594572
timestep 2 inf 0.5342320685198467
timestep 2 inf 0.31037122309302495
timestep 2 inf 0.6129252018504749
timestep 3 1.6053530686616198 0.5342320882470044
timestep 3 0.6252947827329536 0.3103712341835759
timestep 3 1.476997541873412 0.6129252212744569
timestep 4 1.6053530686616198 0.5342319779494393
timestep 4 0.6252947827329536 0.3103712248102074
timestep 4 1.476997541873412 0.6129251821765849
timestep 5 1.6053530686616198 0.549207568593496
timestep 5 0.6252947827329536 0.3398418125862582
timestep 5 1.476997541873412 0.6660121428233544
timestep 6 0.6252947827329536 0.33984182480623804
timestep 6 1.6053530686616198 0.5492075747552134
timestep 6 1.476997541873412 0.6660120281133304
timestep 7 1.3229175843796466 0.549207592835204
timestep 7 0.53645566289522 0.33984181622052884
timestep 7 1.422434250728174 0.6660120355887363
timestep 8 1.067536108010431 ti

/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb.py:364: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


1
timestep 0 inf 0.5342319573135668


/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb.py:364: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:
/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb.py:364: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


11

timesteptimestep  00  infinf  0.61292515681801120.3103712106221212

timestep 1 inf timestep0.3103712645068246 
1 inf 0.6129252961594572
timestep 1 inf 0.5342319390717347
timestep 2 inf 0.5342320685198467
timestep 2 inf 0.31037122309302495
timestep 2 inf 0.6129252018504749
timestep 3 1.6053530686616198 0.5342320882470044
timestep 3 1.476997541873412 0.6129252212744569
timestep 3 0.6252947827329536 0.3103712341835759
timestep 4 1.6053530686616198 0.5342319779494393
timestep 4 1.476997541873412 0.6129251821765849
timestep 4 0.6252947827329536 0.3103712248102074
timestep 5 1.6053530686616198 0.549207568593496
timestep 5 1.476997541873412 0.6660121428233544
timestep 5 0.6252947827329536 0.3398418125862582
timestep 6 1.6053530686616198 0.5492075747552134
timestep 6 1.476997541873412 0.6660120281133304
timestep 6 0.6252947827329536 0.33984182480623804
timestep 7 1.3229175843796466 0.549207592835204
timestep 7 1.422434250728174 0.6660120355887363
timestep 7 0.53645566289522 0.3398418162205

In [10]:
import cvxpy as cp
import numpy as np
import time

class Beamforming():
    def __init__(self, H, max_ant=None, M=1000, noise_var=1, min_snr=1):
        self.N, self.K = H.shape
        
        self.H = H.copy()
        self.max_ant = max_ant
        self.M = M
        self.zero = np.zeros(self.N)
        self.one = np.ones(self.N)
        
        self.c_1 = cp.Parameter()
        self.c_1.value = (1/np.sqrt(min_snr*noise_var))
        self.c_2 = (1/noise_var)

        self.z_constr = cp.Parameter(self.N)
        self.z_mask = cp.Parameter(self.N)
        
        self.W = cp.Variable((self.N, self.K), complex=True, name='W')
        self.z = cp.Variable((self.N), complex=False, name='z')
        self.obj = cp.Minimize(cp.square(cp.norm(self.W, 'fro')))

        self.constraints = []
        for k in range(self.K):
            self.constraints += [self.c_1*cp.real(np.expand_dims(self.H[:,k], axis=0) @ self.W[:,k]) >= cp.norm(cp.hstack((self.c_2*self.W.H @ self.H[:,k], np.ones(1))), 2)]
        self.constraints += [self.z >= self.zero, self.z <= self.one]
        
        self.constraints += [cp.sum(self.z) <= self.max_ant] 
#         self.constraints += [self.z == cp.multiply(self.z_mask,self.z_sol)]
        self.constraints += [cp.multiply(self.z, self.z_mask) == self.z_constr]

#         for n in range(N):
#             if self.z_mask[n]:
#                 self.constraints += [self.z[n] == self.z_constr[n]]
                
        self.constraints += [cp.norm(self.W, 'inf', axis=1) <= self.M*self.z]
        self.prob = cp.Problem(self.obj, self.constraints)


    def solve_beamforming(self, z_mask=None, z_sol=None, W_init=None, z_init=None):
        if W_init is not None:
            self.W.value = W_init.copy()
        if z_init is not None:
            self.z.value = z_init.copy()
        
        self.z_mask.value = z_mask.copy()
        self.z_constr.value = (z_mask*z_sol).copy()
        self.prob.solve(solver=cp.MOSEK, verbose=True, warm_start=True)
        if self.prob.status in ['infeasible', 'unbounded']:
            print('infeasible solution')
            return None, None, np.inf

        return self.z.value, self.W.value, np.linalg.norm(self.W.value, 'fro')**2


# N, K = 8,8
# max_ant = 5
# np.random.seed(150)
# H = np.random.randn(N, K) + 1j*np.random.randn(N, K)
# z_sol = np.random.binomial(size=N, n=1, p= 0.5)
# z_mask = np.random.binomial(size=N, n=1, p=0.2)

# z_mask = np.array([1, 1, 1, 1, 1, 1, 1, 1])
# z_sol = np.array([1, 0, 1, 1, 1, 0, 0, 1])
# # print(z_mask)
# print(z_sol)
# t1 = time.time()
# z, W, obj = solve_beamforming_relaxed(H, max_ant=5, z_sol=z_sol, z_mask=z_mask)
# print("TIME for completion: ", time.time()- t1)

# t1 = time.time()
# z, W, obj = solve_beamforming_relaxed(H, max_ant=5, z_sol=z_sol, z_mask=z_mask, z_init=z, W_init=W)
# print("TIME for completion 2: ", time.time()- t1)

# print(obj)

In [11]:
# from beamforming_test import Beamforming

np.random.seed(seed = 100)
N, K = 8, 3
instances = np.random.randn(1, 2, N, K)
H = instances[0,0,:,:] + 1j*instances[0,1,:,:]

z_mask = np.zeros(N)
z_sol = np.zeros(N)

beam = Beamforming(H, max_ant=3)
t1 = time.time()
a = beam.solve_beamforming(z_mask=z_mask, z_sol=z_sol)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Feb 09 08:43:04 PM: Your problem has 32 variables, 8 constraints, and 17 parameters.
(CVXPY) Feb 09 08:43:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 09 08:43:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 09 08:43:04 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Feb 09 08:43:04 PM: Reduction chain: Complex2Real -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Feb 09 08:43:04 PM: Applying reduction Complex2Real
(CVXPY) Feb 09 08:43:04 PM: Applying reduction 

In [110]:
z_mask =  np.array([0., 0., 0., 0., 0., 1., 0., 0.])
z_sol =  np.array([0.27666557, 0.27658954, 0.27659893, 0.27658924, 0.27666215, 0., 0.27660933, 0.27661012])

b = beam.solve_beamforming(z_mask=z_mask, z_sol=z_sol)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Feb 09 08:41:04 PM: Your problem has 32 variables, 15 constraints, and 9 parameters.
(CVXPY) Feb 09 08:41:04 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 09 08:41:04 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 09 08:41:04 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Feb 09 08:41:04 PM: Finished problem compilation (took 7.279e-03 seconds).
-------------------------------------------------------------------------------
                              

In [97]:
z_mask = np.zeros(N)
z_sol = np.zeros(N)

In [98]:
z_mask

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [93]:
H

array([[-1.74976547+0.18451869j,  0.3426804 +0.9370822j ,
         1.1530358 +0.73100034j],
       [-0.25243604+1.36155613j,  0.98132079-0.32623806j,
         0.51421884+0.05567601j],
       [ 0.22117967+0.22239961j, -1.07004333-1.443217j  ,
        -0.18949583-0.75635231j],
       [ 0.25500144+0.81645401j, -0.45802699+0.75044476j,
         0.43516349-0.45594693j],
       [-0.58359505+1.18962227j,  0.81684707-1.69061683j,
         0.67272081-1.35639905j],
       [-0.10441114-1.23243451j, -0.53128038-0.54443916j,
         1.02973269-0.66817174j],
       [-0.43813562+0.00731456j, -1.11831825-0.61293874j,
         1.61898166+1.29974807j],
       [ 1.54160517-1.73309562j, -0.25187914-0.9833101j ,
        -0.84243574+0.35750775j]])

In [86]:
import time 
N, K = 8,3
max_ant = 5
# np.random.seed(150)

H = np.random.randn(N, K) + 1j*np.random.randn(N, K)
z_sol = np.random.binomial(size=N, n=1, p= 0.1)
z_mask = np.random.binomial(size=N, n=1, p=0.3)

# z_mask = np.array([1, 1, 1, 1, 1, 1, 1, 1])
# z_sol = np.array([1, 0, 1, 1, 1, 0, 0, 1])
# beam = Beamforming(H, max_ant=max_ant)

beam = Beamforming(H, max_ant=max_ant)
t1 = time.time()
beam.solve_beamforming(z_mask=z_mask, z_sol=z_sol)
print('time taken ', time.time()-t1)


                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Feb 09 08:23:56 PM: Your problem has 32 variables, 8 constraints, and 17 parameters.
(CVXPY) Feb 09 08:23:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 09 08:23:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 09 08:23:56 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Feb 09 08:23:56 PM: Reduction chain: Complex2Real -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Feb 09 08:23:56 PM: Applying reduction Complex2Real
(CVXPY) Feb 09 08:23:56 PM: Applying reduction 

In [85]:
t1 = time.time()
z_sol = np.random.binomial(size=N, n=1, p= 0.1)
z_mask = np.random.binomial(size=N, n=1, p=0.3)
# z_sol = np.random.binomial(size=N, n=1, p=0.5)
beam.solve_beamforming(z_mask=z_mask, z_sol=z_sol, W_init=beam.W.value, z_init=beam.z.value)
print('time taken ', time.time()-t1)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Feb 09 08:23:51 PM: Your problem has 32 variables, 8 constraints, and 17 parameters.
(CVXPY) Feb 09 08:23:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 09 08:23:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 09 08:23:51 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Feb 09 08:23:51 PM: Finished problem compilation (took 6.886e-03 seconds).
-------------------------------------------------------------------------------
                              

In [11]:
class BeamformingWithSelectedAntennas():
    def __init__(self, H, max_ant=None, M=1000, noise_var=1, min_snr=1):
        self.N, self.K = H.shape
        
        self.H = H.copy()
        self.max_ant = max_ant
        self.M = M
        self.zero = np.zeros(self.N)
        self.one = np.ones(self.N)
        
        self.c_1 = cp.Parameter()
        self.c_1.value = (1/np.sqrt(min_snr*noise_var))
        self.c_2 = (1/noise_var)
        
        self.z_constr = cp.Parameter(self.N)
        
        self.W = cp.Variable((self.N, self.K), complex=True, name='W')
        self.obj = cp.Minimize(cp.square(cp.norm(self.W, 'fro')))
        
        self.constraints = []
        for k in range(self.K):
            self.constraints += [self.c_1*cp.real(np.expand_dims(self.H[:,k], axis=0) @ self.W[:,k]) >= cp.norm(cp.hstack((self.c_2*self.W.H @ self.H[:,k], np.ones(1))), 2)]
        
        self.constraints += [cp.norm(self.W, 'inf', axis=1) <= self.M*self.z_constr]
        self.prob = cp.Problem(self.obj, self.constraints)
        
        
    def solve_beamforming(self, z=None, W_init=None):
        if W_init is not None:
            self.W.value = W_init.copy()
        
        self.z_constr.value = z.copy()
        self.prob.solve(solver=cp.MOSEK, verbose=True)
        if self.prob.status in ['infeasible', 'unbounded']:
            print('infeasible solution')
            return None, None, np.inf
        
        return self.W.value, np.linalg.norm(self.W.value, 'fro')**2




In [21]:
N, K = 12,6
max_ant = 8
np.random.seed(150)

H = np.random.randn(N, K) + 1j*np.random.randn(N, K)
z_sol = np.random.binomial(size=N, n=1, p= 0.5)
# z_mask = np.random.binomial(size=N, n=1, p=0.9)

# z_mask = np.array([1, 1, 1, 1, 1, 1, 1, 1])
# z_sol = np.array([1, 0, 1, 1, 1, 0, 0, 1])
# beam = Beamforming(H, max_ant=max_ant)

bm = BeamformingWithSelectedAntennas(H, max_ant=max_ant)
t1 = time.time()
out = bm.solve_beamforming(z=z_sol)
print("time: ", time.time()-t1)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Feb 09 03:17:56 PM: Your problem has 72 variables, 7 constraints, and 13 parameters.
(CVXPY) Feb 09 03:17:56 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 09 03:17:56 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 09 03:17:56 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Feb 09 03:17:56 PM: Reduction chain: Complex2Real -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> MOSEK
(CVXPY) Feb 09 03:17:56 PM: Applying reduction Complex2Real
(CVXPY) Feb 09 03:17:56 PM: Applying reduction 

In [20]:
# bm = BeamformingWithSelectedAntennas(H, max_ant=max_ant)
t1 = time.time()
out = bm.solve_beamforming(z=z_sol)
print("time: ", time.time()-t1)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Feb 09 03:17:40 PM: Your problem has 72 variables, 7 constraints, and 13 parameters.
(CVXPY) Feb 09 03:17:40 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 09 03:17:40 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Feb 09 03:17:40 PM: Using cached ASA map, for faster compilation (bypassing reduction chain).
(CVXPY) Feb 09 03:17:40 PM: Finished problem compilation (took 7.732e-03 seconds).
-------------------------------------------------------------------------------
                              

In [19]:
import cvxpy as cp
import numpy as np
from beamforming import solve_beamforming_with_selected_antennas




class SparseIteration():
    def __init__(self, H, noise_var=1, min_snr=1):
        N, K = H.shape
        self.W = cp.Variable((N,K), complex=True)
        self.u = cp.Parameter(N)
        
        self.obj = cp.Minimize((self.u.T @ cp.norm_inf(self.W, axis=1)))

        zero = np.zeros(N)
        one = np.ones(N)
        c_1 = (1/np.sqrt(min_snr*noise_var))
        c_2 = (1/noise_var)

        self.constraints = []
        for k in range(K):
            self.constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ self.W[:,k]) >= cp.norm(cp.hstack((c_2*self.W.H @ H[:,k], np.ones(1))), 2)]

        self.prob = cp.Problem(self.obj, self.constraints)
        
    def solve(self, u):
        self.u.value = u.copy() 
        self.prob.solve(solver=cp.MOSEK, verbose=False)
        
        if self.prob.status in ['infeasible', 'unbounded']:
            print('infeasible solution')
            return None, None, np.inf

        return self.W.value, np.linalg.norm(self.W.value, 'fro')**2


    
class SDPOmar:
    def __init__(self,H, noise_var=1, min_snr=1):
        N, K = H.shape
        self.W = cp.Variable((N,K), complex=True)
        
        self.lmbda = cp.Parameter()
        self.u = cp.Parameter(N)
        
        self.obj = cp.Minimize(cp.square(cp.norm(self.W, 'fro')) + self.lmbda*(self.u.T @ cp.norm_inf(self.W, axis=1)))

        zero = np.zeros(N)
        one = np.ones(N)
        c_1 = (1/np.sqrt(min_snr*noise_var))
        c_2 = (1/noise_var)

        self.constraints = []
        for k in range(K):
            self.constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ self.W[:,k]) >= cp.norm(cp.hstack((c_2*self.W.H @ H[:,k], np.ones(1))), 2)]

        self.prob = cp.Problem(self.obj, self.constraints)
        
    def solve(self, u, lmbda):
        self.u.value = u.copy()
        self.lmbda.value = lmbda
        self.prob.solve(solver=cp.MOSEK, verbose=False)

        if self.prob.status in ['infeasible', 'unbounded']:
            print('infeasible solution')
            return None, np.inf

        return self.W.value, np.linalg.norm(self.W.value, 'fro')**2


def as_omar(H, max_ant=5):
    lmbda_lb = 1e-6
    lmbda_ub = 100
    # global lmbda_lb, lmbda_ub
    N,K = H.shape
    
    sparse_iter = SparseIteration(H)
    sdp_omar = SDPOmar(H)
    
    # step 1
    u = np.zeros((N,1))
    u_new = np.ones((N,1))
    r = 0
    max_iter = 15
    while np.linalg.norm(u-u_new)>0.001 and r < max_iter:
        r += 1
        u = u_new.copy()
        print('shape of u', u.shape)
#         W, _ = sparse_iteration(H, u)
        W, _ = sparse_iter.solve(u)
        a = np.linalg.norm(W, axis=1)
        mask = (a>0.01)*1
        if mask.sum()<= max_ant:
            break
        u_new = 1/(np.linalg.norm(W, axis=1) + 1e-5)
    prelim_mask = mask.copy()

    # step 2
    r = 0
    max_iter = 50
    while mask.sum() != max_ant and r < max_iter:
        r += 1
        # if mask.sum() < max_ant:
        lmbda = lmbda_lb + (lmbda_ub - lmbda_lb)/2
#         W, _ = sdp_omar(H, lmbda, u_new)
        W, _ = sdp_omar.solve(lmbda, u_new)

        a = np.linalg.norm(W, axis=1)
        mask = (a>0.01)*1
        print('iteration {}'.format(r), lmbda, mask.sum(), lmbda_lb, lmbda_ub)
        if mask.sum() == max_ant:
            break
        elif mask.sum() > max_ant:
            lmbda_lb = lmbda
        elif mask.sum() < max_ant:
            lmbda_ub = lmbda
    if mask.sum()>max_ant:
        mask = prelim_mask.copy()    
    print('num selected antennas', mask.sum())

    # step 3
    W, obj = solve_beamforming_with_selected_antennas(H, mask)
    print(obj)
    return obj.copy(), mask.copy()


if __name__=='__main__':
    N, K = 8,3
    max_ant = 5
    H = np.random.randn(N, K) + 1j*np.random.randn(N, K)
    as_omar(H, max_ant=max_ant)
    # W, obj = sdp_omar(H, 50, np.ones((N,1)))
    # print(np.linalg.norm(W, axis=1), obj)



shape of u (8, 1)


ValueError: Invalid dimensions (8, 1) for Parameter value.